# Monitoreo de variables hidrológicas / hidrogeológicas / climatológicas derivadas de Percepción Remota y Productos Globales para acuíferos de México

## Descripción

Esta es una notebook para el monitoreo cuantitativo de variables hidrológicas / hidrogeológicas / climatológicas de México utilizando imágenes de diversas bases de datos desde Google Earth Engine.

*Material didáctico que para obtener el título de Ingeniera Geofísica*

Google ha desarrollado Google Earth Engine, una plataforma que cuenta con un extenso catálogo de información derivada de diferentes misiones satelitales. La API para el lenguaje de programación Python permite acceder a la información de imágenes satelitales, desacargando y graficando los datos que se capturan en series de tiempo útiles en el estudio de la calidad de los suelos, sequias, identificar zonas de máxima explotación; por ejemplo: agrícola, ganadera, entre otras. 

## Acerca de los autores

- **Maria Isabel Celso Crescencio**: Ing. Geofísica por la Facultad de Ingeniería de la UNAM

- **Saúl Arciniega Esparza**: Profesor Asociado C de TC, Facultad de Ingeniería de la UNAM

In [1]:
# @title Importar librerias
# Importación de módulos
import json
import datetime
from time import sleep
import numpy as np
import pandas as pd
import geemap.foliumap as geemap
import eemont
import ee

from pprint import pprint

import plotly.offline as po
import plotly.express as pe
import plotly.graph_objects as go

import panel as pn
from io import StringIO
from IPython.display import display

pn.extension('plotly')
pn.extension(design='material')

import warnings
warnings.filterwarnings("ignore")

## Acceder a _Earth Engine_

### Instrucciones:

1. Ejecutar la celda con el título _"Ingresar a Google Earth Engine"_.
2. Acceder al link.
3. Seleccionar la cuenta _Google_ desde la que accesará. $\rightarrow$ Continuar.
4. Seleccionar la casilla _use **read-only scopes**_, posteriormente seleccionar _**GENERAR TOKEN**_.
5. Seleccionar cuenta _Google_ para acceder a _Earth Engine Notebook Client_.
6. Aparecerá un mensaje que indica que la aplicación no esta verificada por _Google_, seleccionar **Continuar**.
7. Seleccionar las dos casillas para permitir el acceso a _Earth Engine Notebook Client_. $\rightarrow$ Continuar.
8. Copiar el código de autorización proporcionado.
9. Pegar el código en la casilla que se ubica debajo del link.

![Gee_guia](https://upload.wikimedia.org/wikipedia/commons/2/2f/Gee_guia.png)


In [2]:
# @title Autenticación para inicializar Google Earth Engine API
# Autenticación y credenciales para inicializar 
ee.Authenticate(force=True)

Enter verification code:  4/1AQSTgQEAaqgmYayVpPO3hbKksmaAPS5xc_XL9tnOIrZyQCZm6ZZIrcqdICs



Successfully saved authorization token.


In [3]:
# @title Inicialización
# Verificación de que existen credenciales válidas
ee.Initialize()

In [4]:
#@title Importar la capa de acuíferos
# Construcción de mapa con datos geograficos
layer_file = r"./data/Acuiferos.shp"
capa = geemap.shp_to_ee(layer_file)

In [5]:
#@title Infomacion de la capa y colecciones

# Extracción de la información de la capa
capa_atributos = capa.getInfo()
columna, sy_in = capa_atributos["columns"]
campos = [item["properties"][columna] for item in capa_atributos["features"]]
campos.sort()

# Diccionario de colecciones
colecciones = {
    "3B43V7 - Precipitación": ('TRMM/3B43V7', 'precipitation', 'mm', 720, "mean", "Mensual"),
    "CHIRPS - Precipitación": ('UCSB-CHG/CHIRPS/DAILY', 'precipitation', 'mm', 1, "sum", "Diario"),
    "GLDAS - AvgSurfT": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'AvgSurfT_tavg', 'K', 1, "mean", "Diario"),
    "GLDAS - CanopInt": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'CanopInt_tavg', 'mm', 1, "mean", "Diario"),
    "GLDAS - ECanop": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'ECanop_tavg', 'mm', 86400, "mean", "Diario"),
    "GLDAS - ESoil": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'ESoil_tavg', 'mm', 86400, "mean", "Diario"),
    "GLDAS - EvapSnow": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'EvapSnow_tavg', 'mm', 86400, "mean", "Diario"),
    "GLDAS - Evap": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'Evap_tavg', 'mm', 86400, "mean", "Diario"),
    "GLDAS - TVeg": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'TVeg_tavg', 'mm', 86400, "mean", "Diario"),
    "GLDAS - GWS": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'GWS_tavg', 'mm', 1, "mean", "Diario"),
    "GLDAS - TWS": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'TWS_tavg', 'mm', 1, "mean", "Diario"),
    "GLDAS - SoilMoist_P": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'SoilMoist_P_tavg', 'mm', 1, "mean", "Diario"),
    "GLDAS - SoilMoist_RZ": ('NASA/GLDAS/V022/CLSM/G025/DA1D', 'SoilMoist_RZ_tavg', 'mm', 1, "mean", "Diario"),
    "DAYMET_V4 - tmax": ('NASA/ORNL/DAYMET_V4', 'tmax', '°C', 1, "mean", "Diario"),
    "DAYMET_V4 - tmin": ('NASA/ORNL/DAYMET_V4', 'tmin', '°C', 1, "mean", "Diario"),
    "ERA5 - TMdA": ('ECMWF/ERA5/DAILY', 'mean_2m_air_temperature', 'K', 1, "mean", "Diario"),
    "ERA5 - TMnA": ('ECMWF/ERA5/DAILY', 'minimum_2m_air_temperature', 'K', 1, "mean", "Diario"),
    "ERA5 - TMxA": ('ECMWF/ERA5/DAILY', 'maximum_2m_air_temperature', 'K', 1, "mean", "Diario"),
    "ERA5 - TP": ('ECMWF/ERA5/DAILY', 'total_precipitation', 'mm', 1000, "sum", "Diario"),
    "ERA5 - SP": ('ECMWF/ERA5/DAILY', 'surface_pressure', 'Pa', 1, "mean", "Diario"),
    "IMERG_MONTHLY_V06 - Precipitación": ('NASA/GPM_L3/IMERG_MONTHLY_V06', 'precipitation', 'mm', 720, "mean", "Mensual"),
    "MOD16A2-ET": ('MODIS/061/MOD16A2', 'ET', 'mm', 0.1, "mean", "Semanal"),
    "MOD13A2 - EVI": ('MODIS/061/MOD13A2', 'EVI', ' ', 0.0001, "mean", "Mensual"),
    "MCD15A3H - Lai": ('MODIS/061/MCD15A3H', 'Lai', ' ', 0.1, "mean", "Semanal"),
    "MOD13A2 - NDVI": ('MODIS/061/MOD13A2', 'NDVI', ' ', 0.0001, "mean", "Mensual"),
    "MOD16A2-PET": ('MODIS/061/MOD16A2', 'PET', 'mm', 0.1, "mean", "Semanal"),
    "TERRACLIMATE - aet": ('IDAHO_EPSCOR/TERRACLIMATE', 'aet', 'mm', 0.1, "sum", "Mensual"),
    "TERRACLIMATE - soil": ('IDAHO_EPSCOR/TERRACLIMATE', 'soil', 'mm', 0.1, "mean", "Mensual"),
    "TERRACLIMATE - tmmn": ('IDAHO_EPSCOR/TERRACLIMATE', 'tmmn', '°C', 0.1, "mean", "Mensual"),
    "TERRACLIMATE - tmmx": ('IDAHO_EPSCOR/TERRACLIMATE', 'tmmx', '°C', 0.1, "mean", "Mensual"),
    "PERSIANN-CDR -  Precipitación": ('NOAA/PERSIANN-CDR', 'precipitation', 'mm', 1, "sum", "Diario")
}

In [6]:
# @title Funciones de procesamiento

# Conversión de las series temporales por región 
# a series de tiempo en la librería pandas 
def gee_ts_to_serie(ts, var):
    info = ts.getInfo()
    data = []
    for feat in info["features"]:
        d = feat["properties"]
        data.append([d["date"], d[var]])
    df = pd.DataFrame(data, columns=["date", var])
    df.set_index("date", inplace=True)
    df.index = pd.to_datetime(df.index)
    df[df == -9999] = np.nan
    df = df.dropna()
    dates = pd.to_datetime([f"{x.year}-{x.month}-{x.day}" for x in df.index])
    df.index = dates
    df.sort_index(inplace=True)
    return df

# Obtención de las series por región para la colección de imágenes y geometría dadas 
def collection_timeserie(collection, geom, band, scale=30, reducer="mean"):
    if reducer == "mean":
        reducer = ee.Reducer.mean()
    elif reducer == "sum":
        reducer = ee.Reducer.sum()
    elif reducer == "count":
        reducer = ee.Reducer.count()
    elif reducer == "covariance":
        reducer = ee.Reducer.covariance()
    elif reducer == "std":
        reducer = ee.Reducer.stdDev()
    elif reducer == "min":
        reducer = ee.Reducer.min(1)
    elif reducer == "max":
        reducer = ee.Reducer.max(1)

    ts = collection.getTimeSeriesByRegion(
        reducer = [reducer],
        geometry = geom,
        bands = [band],
        scale = scale,
        maxPixels= 1e10
    )
    serie = gee_ts_to_serie(ts, band).loc[:, band]
    return serie

# Descarga de serie definida por acuífero y base de datos
def get_data(Clave_acuifero, Base_de_datos, fecha_in, fecha_fin):
    attributes = colecciones[Base_de_datos]
    geom = capa.filter(ee.Filter.eq(columna, Clave_acuifero))

    yearst=int(fecha_in[0:4])
    yearnd=int(fecha_fin[0:4])
    monthst=(fecha_in[5:7])
    monthnd=(fecha_fin[5:7])
    dayst=(fecha_in[8:10])
    daynd=(fecha_fin[8:10])

    escala    = 5000  # en m
    presicion = 5     # numero de decimales a guardar
    dy        = 5     # numero de años a descargar en una peticion
    year1 = np.arange(yearst, yearnd+1, dy)
    year2 = year1 + dy
    if year2[-1] > yearnd: year2[-1] = yearnd
    years = list(zip(year1, year2))

    serie=[]
    for y1, y2 in years:
        start = f"{y1}-{monthst}-{dayst}"
        if y2 < yearnd:
            date2 = pd.to_datetime(f"{y2}-{monthst}-{dayst}") - pd.Timedelta("1d")
            end = date2.strftime("%Y-%m-%d")
        else:
            end = f"{y2}-{monthnd}-{daynd}"
        dataset = (ee.ImageCollection(attributes[0])
                   .filterBounds(geom)
                   .filterDate(start, end))
        try:
            ts = collection_timeserie(dataset, geom, attributes[1], escala) * attributes[3]
        except:
            sleep(30)
            ts = collection_timeserie(dataset, geom, attributes[1], escala) * attributes[3]
        sleep(1)
        serie.append(ts)

    pd_serie = pd.concat(serie)

    return pd_serie, attributes

# Remuestreo de serie temporal
def agg_timeserie(serie, Timestep, agg="sum", freq="Diario"):
    if Timestep == "Mensual":
        if agg == "sum":
            pd_serie = serie.resample("1M").sum()
        else:
            pd_serie = serie.resample("1M").mean()
    elif Timestep == "Anual":
        if agg == "sum":
            pd_serie = serie.resample("1Y").sum()
        else:
            pd_serie = serie.resample("1Y").mean()
    elif Timestep == "Diario":
        if freq != "Diario":
            if agg == "sum":
                pd_serie = serie.resample("1M").sum()
            else:
                pd_serie = serie.resample("1M").mean()
        else:
            if agg == "sum":
                pd_serie = serie.resample("1D").sum()
            else:
                pd_serie = serie.resample("1D").mean()
    return pd_serie

# Consultar datos

Instrucciones:

1. Selecciona la clave del acuífero

2. Selecciona una colección y variable

3. _Click_ en **Run Interactive**

## Colecciones

| **Base de datos - Nombre** | **Colección** | **Variable** | **Descripción** |
| :-: |:-: | :-: | :-: |
| 3B43V7 - Precipitación | TRMM/3B43V7 | Velocidad de precipitación | Precipitación estimada por microondas/IR de TRMM |   
| CHIRPS - Precipitación | UCSB-CHG/CHIRPS/DAILY | Velocidad de precipitación | Precipitación |
| GLDAS - AvgSurfT | NASA/GLDAS/V022/CLSM/G025/DA1D | Temperatura | Temperatura media de la superficie de GLDAS |
| GLDAS - CanopInt | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial | Agua superficial de la cubierta vegetal de GLDAS |
| GLDAS - ECanop | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial por unidad de tiempo | Evaporación de agua de las cimas de GLDAS |
| GLDAS - ESoil | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial por unidad de tiempo | Evaporación de agua del suelo desnudo de GLDAS |
| GLDAS - EvapSnow | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial por unidad de tiempo | Evaporación de la nieve de GLDAS |
| GLDAS - Evap | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial por unidad de tiempo | Evapotranspiración de GLDAS |
| GLDAS - TVeg | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial por unidad de tiempo | Transpiración de GLDAS |
| GLDAS - GWS | NASA/GLDAS/V022/CLSM/G025/DA1D | Altura de almacenamiento | Almacenamiento de agua subterránea de GLDAS |
| GLDAS - TWS | NASA/GLDAS/V022/CLSM/G025/DA1D | Altura de almacenamiento | Almacenamiento de agua terrestre de GLDAS |
| GLDAS - SoilMoist_P | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial | Perfil de humedad del suelo de GLDAS |
| GLDAS - SoilMoist_RZ | NASA/GLDAS/V022/CLSM/G025/DA1D | Densidad superficial | Humedad del suelo de zona con raices de GLDAS |
| DAYMET_V4 - tmax | NASA/ORNL/DAYMET_V4 | Temperatura | Temperatura máxima del aire a 2m |
| DAYMET_V4 - tmin | NASA/ORNL/DAYMET_V4 | Temperatura | Temperatura mínima del aire a 2m |
| ERA5 - TMdA | ECMWF/ERA5/DAILY | Temperatura | Temperatura diaria media del aire a 2m de altura de ECMWF|
| ERA5 - TMnA | ECMWF/ERA5/DAILY | Temperatura | Temperatura diaria mínima del aire a 2m de altura de ECMWF|
| ERA5 - TMxA | ECMWF/ERA5/DAILY | Temperatura | Temperatura diaria máxima del aire a 2m de altura de ECMWF|
| ERA5 - TP | ECMWF/ERA5/DAILY | Altura de la capa de agua | Precipitación de ECMWF|
| ERA5 - SP | ECMWF/ERA5/DAILY | Presión | Presión superficial de ECMWF|
| IMERG_MONTHLY_V06 - Precipitación | NASA/GPM_L3/IMERG_MONTHLY_V06 | Velocidad de precipitación | Estimación de la precipitación por satélite y manómetro de GPM |
| MOD16A2-ET | MODIS/061/MOD16A2 | Densidad superficial por unidad de tiempo | Evapotranspiración de MODIS |
| MOD13A2 - EVI | MODIS/061/MOD13A2 | Vegetación | Índice de vegetación mejorado de MODIS |
| MCD15A3H - Lai | MODIS/061/MCD15A3H | Vegetación | Superficie de una hoja verde de MODIS |
| MOD13A2 - NDVI | MODIS/061/MOD13A2 | Vegetación | Índice de vegetación normalizada de MODIS |
| MOD16A2-PET | MODIS/061/MOD16A2 | Densidad superficial por unidad de tiempo | Evapotranspiración potencial de MODIS |
| TERRACLIMATE - aet | IDAHO_EPSCOR/TERRACLIMATE | Altura de la capa | Evapotranspiración de Idaho EPSCoR |
| TERRACLIMATE - soil | IDAHO_EPSCOR/TERRACLIMATE | Altura de la capa | Humedad del suelo de Idaho EPSCoR |
| TERRACLIMATE - tmmn | IDAHO_EPSCOR/TERRACLIMATE | Temperatura | Temperatura mínima de Idaho EPSCoR |
| TERRACLIMATE - tmmx | IDAHO_EPSCOR/TERRACLIMATE | Temperatura | Temperatura máxima de Idaho EPSCoR |
| PERSIANN-CDR -  Precipitación | NOAA/PERSIANN-CDR | Altura de la capa de agua | Precipitación diaria estimada de NOAA |

In [7]:
# @title Crear App

# Datos globales
query = {"serie": None, "attributes": None}

# Herramientas interactivas
clave_a = pn.widgets.Select(
    name="Clave de acuífero",
    value=campos[0],
    options=campos
)
base_deDatos = pn.widgets.Select(
    name="Base de datos",
    value=list(colecciones.keys())[0],
    options=list(colecciones.keys())
)
Timestep = pn.widgets.Select(
    name="Agregación",
    value="Diario",
    options=["Diario", "Mensual", "Anual"]
)
fecha_in = pn.widgets.DatePicker(
    name="Fecha inicial",
    value=datetime.date(2010, 1, 1)
)
fecha_fin = pn.widgets.DatePicker(
    name="Fecha final",
    value=datetime.datetime.now()
)
download_text = pn.pane.Markdown("")
guardar_como = pn.widgets.TextInput(name="Guardar como", placeholder="Nombre de archivo de descarga...")

figure = go.Figure(
    data=go.Scatter(x=[], y=[])
)
fig_responsive = pn.pane.Plotly(figure, sizing_mode='stretch_width')

# Funciones de llamada
def on_button_clicked(b):
    button.name = "Procesando..."
    download_text.object = "Generando base de datos...."
    datos_downloaded, attributes = get_data(clave_a.value, base_deDatos.value, str(fecha_in.value), str(fecha_fin.value))
    query["serie"] = datos_downloaded
    query["attributes"] = attributes
    datos = agg_timeserie(datos_downloaded, Timestep.value, agg=attributes[4], freq=attributes[5])
    # Actualizar figura
    figure = go.Figure(
        data=go.Scatter(
            x=datos.index,
            y=datos,
            mode="lines"
        )
    )
    figure.update_yaxes(title_text='Fecha')
    figure.update_yaxes(title_text=attributes[2])
    figure.update_layout(
        height=400,
        title=f'Colección "{base_deDatos.value}" del acuifero {clave_a.value}',
        title_x=0.5,
        title_y=0.95,
        showlegend=False,
    )
    fig_responsive.object = figure
    # Actualizar etiquetas
    button.name = "Consultar datos"
    button_download.disabled = False
    download_text.object = ""

def update_timestep(timestep):
    download_text.object = ""
    datos_downloaded = query["serie"]
    attributes = query["attributes"]
    if datos_downloaded is not None:
        datos = agg_timeserie(datos_downloaded, timestep, agg=attributes[4], freq=attributes[5])
        figure = go.Figure(
            data=go.Scatter(
                x=datos.index,
                y=datos,
                mode="lines"
            )
        )
        figure.update_yaxes(title_text='Fecha')
        figure.update_yaxes(title_text=attributes[2])
        figure.update_layout(
                height=400,
                title=f'Colección "{base_deDatos.value}" del acuifero {clave_a.value}',
                title_x=0.5,
                title_y=0.95,
                showlegend=False,
            )
        fig_responsive.object = figure

def download_serie():
    presicion = 5
    saveas = guardar_como.value
    if len(saveas) == 0:
        saveas = f"{clave_a.value}-{base_deDatos.value}"
    button_download.filename = f"{saveas}.csv"
    if query["serie"] is not None:
        datos = agg_timeserie(query["serie"], Timestep.value, agg=query["attributes"][4], freq=query["attributes"][5])
        sio = StringIO()
        datos.round(presicion).to_csv(sio)
        sio.seek(0)
        return sio

# Controles interactivos
button = pn.widgets.Button(name="Consultar datos", button_type="primary")
button.on_click(on_button_clicked)
button_download = pn.widgets.FileDownload(
    label="Descargar Series",
    callback=pn.bind(download_serie),
    button_type="primary",
    disabled=True
)

# App principal
app = pn.Column(
    pn.Row(clave_a, base_deDatos),
    pn.Row(fecha_in, fecha_fin),
    pn.Row(Timestep, guardar_como),
    pn.Row(button, button_download),
    download_text,
    fig_responsive,
    pn.bind(update_timestep, Timestep),
    sizing_mode='stretch_width',   
)

app

Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Select(design=<class 'panel.theme.materi..., name='Clave de acuífero', options=[101, 102, 103, ...], value=101)
        [1] Select(design=<class 'panel.theme.materi..., name='Base de datos', options=['3B43V7 - Precipitación',...], value='3B43V7 - Precipitación')
    [1] Row(design=<class 'panel.theme.materi...)
        [0] DatePicker(design=<class 'panel.theme.materi..., name='Fecha inicial', value=datetime.date(2010, 1, 1))
        [1] DatePicker(design=<class 'panel.theme.materi..., name='Fecha final', value=datetime.date(2025, ...)
    [2] Row(design=<class 'panel.theme.materi...)
        [0] Select(design=<class 'panel.theme.materi..., name='Agregación', options=['Diario', 'Mensual', ...], value='Diario')
        [1] TextInput(design=<class 'panel.theme.materi..., name='Guardar como', placeholder='Nombre de archivo d...)
    [3] Row(design=<class 'panel.theme.materi...)
        [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Consultar datos')
        [1] FileDownload(button_type='primary', callback=<function bind.<locals>.wr..., design=<class 'panel.theme.materi..., disabled=True, label='Descargar Series')
    [4] Markdown(str, design=<class 'panel.theme.materi...)
    [5] Plotly(Figure, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [6] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)